In [ ]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [ ]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


# Check the iteration number

In [ ]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

# Check the drug

In [ ]:
drug = 'ACE'
print("This is the drug:", drug)

# Optimizer initialization (N/A)

# Generate recommendations

In [ ]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

In [ ]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

In [ ]:
mask = (unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']] == 0).all(axis=1)

# print the rows that are about to change
print("Rows to be updated (before):")
print(unlabeled_trial.loc[mask])

# set Water to 100 on those rows
unlabeled_trial.loc[mask, 'Water'] = 100

# print the rows after the update
print("\nRows after update:")
print(unlabeled_trial.loc[mask])

In [ ]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

In [ ]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

In [ ]:
unlabeled_trial
unlabeled_trial.describe()

In [ ]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

In [ ]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

In [ ]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [ ]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [ ]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [ ]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [ ]:
updated_script

# Perform experiment

# Results

In [ ]:
results = calc.parameters_and_complexity(iteration)
results

In [ ]:
size_raw = calc.size_raw(iteration)
size_raw

In [ ]:
size_processed = calc.process_formulations(size_raw)
size_processed

In [ ]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

In [ ]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

In [ ]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

In [ ]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

In [ ]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [ ]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [ ]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

In [ ]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [ ]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [ ]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)